In [1]:
import torch
import cv2
import numpy as np
from PIL import Image
import imageio
import os
from pathlib import Path
import gc
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video
import torchvision.transforms as transforms
from IPython.display import HTML, display
import ipywidgets as widgets
from io import BytesIO
import base64

In [10]:
# سلول 3: تنظیمات قابل تغییر - 🔧 این سلول را می‌توانید تغییر دهید
class VideoConfig:
    # تنظیمات رزولیوشن
    OUTPUT_WIDTH = 1080      # عرض نهایی
    OUTPUT_HEIGHT = 1920     # ارتفاع نهایی
    
    # تنظیمات زمان
    DURATION_SECONDS = 5   # مدت زمان ویدیو (ثانیه)
    FPS = 6                  # فریم در ثانیه
    
    # تنظیمات حافظه (برای RTX 4050)
    ENABLE_CPU_OFFLOAD = True
    ENABLE_MEMORY_EFFICIENT = True
    USE_FP16 = True
    
    # تنظیمات کیفیت
    UPSCALE_METHOD = "realesrgan"  # یا "bicubic" برای سرعت بیشتر
    GUIDANCE_SCALE = 10.0
    MOTION_BUCKET_ID = 127
    NOISE_AUG_STRENGTH = 0.02
    
    # تنظیمات دیگر
    SEED = 42
    SAVE_FRAMES = False      # ذخیره فریم‌های جداگانه
    OUTPUT_FORMAT = "mp4"

config = VideoConfig()

# محاسبه تعداد فریم‌ها
TOTAL_FRAMES = int(config.DURATION_SECONDS * config.FPS)
print(f"📊 تنظیمات فعلی:")
print(f"   رزولیوشن نهایی: {config.OUTPUT_WIDTH}x{config.OUTPUT_HEIGHT}")
print(f"   مدت زمان: {config.DURATION_SECONDS} ثانیه")
print(f"   تعداد فریم: {TOTAL_FRAMES}")
print(f"   FPS: {config.FPS}")

📊 تنظیمات فعلی:
   رزولیوشن نهایی: 1080x1920
   مدت زمان: 5 ثانیه
   تعداد فریم: 30
   FPS: 6


In [3]:
# سلول 4: لود مدل SVD-XT
def load_model():
    print("🔄 در حال لود مدل SVD-XT...")
    
    # تنظیمات torch
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    # لود pipeline با progress bar
    print("📥 دانلود مدل (ممکن است چند دقیقه طول بکشد)...")
    pipe = StableVideoDiffusionPipeline.from_pretrained(
        "stabilityai/stable-video-diffusion-img2vid-xt",
        torch_dtype=torch.float16 if config.USE_FP16 else torch.float32,
        variant="fp16" if config.USE_FP16 else None
    )
    
    # تنظیمات حافظه
    if config.ENABLE_CPU_OFFLOAD:
        pipe.enable_model_cpu_offload()
        print("✅ CPU offloading فعال شد")
    
    if config.ENABLE_MEMORY_EFFICIENT:
        # فقط attention slicing (vae_slicing در SVD موجود نیست)
        pipe.enable_attention_slicing()
        print("✅ Memory efficient attention فعال شد")
        
        # تنظیمات اضافی برای صرفه‌جویی حافظه
        try:
            pipe.enable_sequential_cpu_offload()
            print("✅ Sequential CPU offload فعال شد")
        except:
            print("⚠️ Sequential CPU offload پشتیبانی نمی‌شود")
    
    # انتقال به GPU
    if not config.ENABLE_CPU_OFFLOAD:
        pipe = pipe.to("cuda")
    
    print("✅ مدل با موفقیت لود شد")
    return pipe

# لود مدل
pipe = load_model()

Couldn't connect to the Hub: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/stabilityai/stable-video-diffusion-img2vid-xt (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x753798a95220>: Failed to establish a new connection: [Errno 111] Connection refused')))"), '(Request ID: e4bc84d4-d103-41cc-bf43-49f9a7503303)').
Will try to load from local cache.


🔄 در حال لود مدل SVD-XT...
📥 دانلود مدل (ممکن است چند دقیقه طول بکشد)...


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

✅ CPU offloading فعال شد
✅ Memory efficient attention فعال شد
✅ Sequential CPU offload فعال شد
✅ مدل با موفقیت لود شد


In [4]:
# سلول 5: تابع Upscaling
def upscale_frames(frames, target_width, target_height, method="realesrgan"):
    """
    Upscale فریم‌ها به رزولیوشن هدف
    """
    upscaled_frames = []
    
    if method == "realesrgan":
        try:
            from realesrgan import RealESRGANer
            from basicsr.archs.rrdbnet_arch import RRDBNet
            
            # تنظیم Real-ESRGAN
            model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
            upsampler = RealESRGANer(
                scale=4,
                model_path='https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth',
                model=model,
                tile=0,
                tile_pad=10,
                pre_pad=0,
                half=True if config.USE_FP16 else False
            )
            
            print("🔄 Real-ESRGAN upscaling در حال انجام...")
            for i, frame in enumerate(frames):
                frame_np = np.array(frame)
                output, _ = upsampler.enhance(frame_np, outscale=4)
                
                # تغییر اندازه به رزولیوشن هدف
                output = cv2.resize(output, (target_width, target_height), interpolation=cv2.INTER_LANCZOS4)
                upscaled_frames.append(Image.fromarray(output))
                
                if i % 5 == 0:
                    print(f"   فریم {i+1}/{len(frames)} پردازش شد")
            
        except Exception as e:
            print(f"⚠️ Real-ESRGAN خطا داد، از bicubic استفاده می‌شود: {e}")
            method = "bicubic"
    
    if method == "bicubic":
        print("🔄 Bicubic upscaling در حال انجام...")
        for i, frame in enumerate(frames):
            frame_resized = frame.resize((target_width, target_height), Image.LANCZOS)
            upscaled_frames.append(frame_resized)
            
            if i % 10 == 0:
                print(f"   فریم {i+1}/{len(frames)} پردازش شد")
    
    return upscaled_frames


In [5]:
# سلول 6: تابع اصلی تولید ویدیو
def generate_video(image_path, output_path="output_video.mp4"):
    """
    تولید ویدیو از روی عکس ورودی
    """
    try:
        # لود و آماده‌سازی عکس
        print("🔄 در حال لود عکس...")
        image = load_image(image_path)
        
        # تغییر اندازه عکس به ورودی مناسب SVD (1024x576)
        image = image.resize((1024, 576), Image.LANCZOS)
        
        # تنظیم generator برای نتایج قابل تکرار
        generator = torch.manual_seed(config.SEED) if config.SEED else None
        
        # تولید ویدیو
        print("🎬 در حال تولید ویدیو...")
        print(f"   تعداد فریم: {TOTAL_FRAMES}")
        
        # آزاد کردن حافظه
        gc.collect()
        torch.cuda.empty_cache()
        
        # تولید فریم‌ها
        frames = pipe(
            image,
            decode_chunk_size=8,
            generator=generator,
            motion_bucket_id=config.MOTION_BUCKET_ID,
            noise_aug_strength=config.NOISE_AUG_STRENGTH,
            num_frames=TOTAL_FRAMES,
        ).frames[0]
        
        print(f"✅ {len(frames)} فریم تولید شد")
        
        # Upscaling به رزولیوشن نهایی
        if config.OUTPUT_WIDTH != 1024 or config.OUTPUT_HEIGHT != 576:
            print("🔄 در حال upscaling...")
            frames = upscale_frames(frames, config.OUTPUT_WIDTH, config.OUTPUT_HEIGHT, config.UPSCALE_METHOD)
        
        # ذخیره فریم‌های جداگانه (اختیاری)
        if config.SAVE_FRAMES:
            frames_dir = Path(output_path).stem + "_frames"
            os.makedirs(frames_dir, exist_ok=True)
            for i, frame in enumerate(frames):
                frame.save(f"{frames_dir}/frame_{i:04d}.png")
            print(f"✅ فریم‌ها در {frames_dir} ذخیره شدند")
        
        # تبدیل به ویدیو
        print("🎥 در حال ذخیره ویدیو...")
        
        # آماده‌سازی فریم‌ها برای imageio
        frames_array = []
        for frame in frames:
            frames_array.append(np.array(frame))
        
        # ذخیره ویدیو
        imageio.mimsave(
            output_path,
            frames_array,
            fps=config.FPS,
            quality=8,
            codec='libx264'
        )
        
        print(f"✅ ویدیو با موفقیت ذخیره شد: {output_path}")
        
        # آزاد کردن حافظه
        del frames, frames_array
        gc.collect()
        torch.cuda.empty_cache()
        
        return output_path
        
    except Exception as e:
        print(f"❌ خطا در تولید ویدیو: {e}")
        return None

In [6]:
# سلول 7: اجرای نهایی - آپلود عکس و تولید ویدیو
def create_video_interface():
    """
    ایجاد رابط کاربری برای آپلود عکس
    """
    print("📱 رابط کاربری آماده است")
    print("=" * 50)
    
    # widget برای آپلود فایل
    file_upload = widgets.FileUpload(
        accept='image/*',
        multiple=False,
        description='آپلود عکس'
    )
    
    # widget برای نام فایل خروجی
    output_name = widgets.Text(
        value='my_video.mp4',
        description='نام فایل:'
    )
    
    # دکمه تولید
    generate_button = widgets.Button(
        description='🎬 تولید ویدیو',
        button_style='success',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # نمایش وضعیت
    status_output = widgets.Output()
    
    def on_generate_click(b):
        with status_output:
            status_output.clear_output()
            
            # چک کردن آپلود فایل (روش‌های مختلف برای نسخه‌های مختلف)
            uploaded_files = None
            try:
                if hasattr(file_upload, 'value') and file_upload.value:
                    uploaded_files = file_upload.value
                elif hasattr(file_upload, 'data') and file_upload.data:
                    uploaded_files = file_upload.data
                else:
                    print("❌ لطفاً ابتدا عکس را آپلود کنید")
                    return
            except:
                print("❌ لطفاً ابتدا عکس را آپلود کنید")
                return
            
            if not uploaded_files:
                print("❌ لطفاً ابتدا عکس را آپلود کنید")
                return
            
            # ذخیره عکس آپلود شده
            try:
                # روش 1: نسخه جدید
                if isinstance(uploaded_files, dict):
                    uploaded_file = list(uploaded_files.values())[0]
                    file_content = uploaded_file['content']
                # روش 2: نسخه قدیمی
                elif isinstance(uploaded_files, tuple):
                    uploaded_file = uploaded_files[0]
                    file_content = uploaded_file['content']
                # روش 3: نسخه خیلی جدید
                elif isinstance(uploaded_files, list):
                    uploaded_file = uploaded_files[0]
                    file_content = uploaded_file['content']
                else:
                    uploaded_file = uploaded_files
                    file_content = uploaded_file['content']
                
                temp_image_path = "temp_input.jpg"
                
                with open(temp_image_path, "wb") as f:
                    f.write(file_content)
                
                print("🚀 شروع تولید ویدیو...")
                result = generate_video(temp_image_path, output_name.value)
                
                if result:
                    print("🎉 ویدیو با موفقیت تولید شد!")
                    print(f"📁 فایل: {result}")
                    
                    # نمایش اطلاعات فایل
                    file_size = os.path.getsize(result) / (1024 * 1024)  # MB
                    print(f"📊 حجم فایل: {file_size:.2f} MB")
                    
                    # پخش ویدیو در نوت‌بوک
                    display(HTML(f"""
                    <video width="400" controls>
                        <source src="{result}" type="video/mp4">
                        مرورگر شما از تگ video پشتیبانی نمی‌کند.
                    </video>
                    """))
                
                # حذف فایل موقت
                if os.path.exists(temp_image_path):
                    os.remove(temp_image_path)
                    
            except Exception as e:
                print(f"❌ خطا در پردازش فایل: {e}")
                print("🔍 در حال تست روش‌های مختلف...")
                
                # debug info
                print(f"نوع uploaded_files: {type(uploaded_files)}")
                if hasattr(uploaded_files, '__dict__'):
                    print(f"attributes: {dir(uploaded_files)}")
                
                return
    
    generate_button.on_click(on_generate_click)
    
    # نمایش رابط
    display(widgets.VBox([
        widgets.HTML("<h3>🎬 تولید ویدیو با SVD-XT</h3>"),
        file_upload,
        output_name,
        generate_button,
        status_output
    ]))


In [11]:
# اجرای رابط کاربری
create_video_interface()

📱 رابط کاربری آماده است
